In [481]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import HTML, Javascript, display, clear_output

# counter variables
num_sim = 1000
time_count = 0
repayment_per_month = []
monthly_payarray = []
status_count = []
snapshot_df = pd.DataFrame()

# independent variables
num_stu = 0
pay_cap = 0
pay_terms = 0
exp_terms = 0
exp_multiple = 0
min_dis = 0
max_dis = 0
min_income = 0
income_growth_rate = 0
grad_rate_4year = 0
total_disbursement = 0
economy = 0

#vocational independent variables
v_level = 0
v_field = 0
v_terms = 0
v_target_roi = 0

In [489]:

# info tab

info = widgets.HTML(value=
                     '''
                     ******************************<b> ISA Fund Simulation </b>******************************
                     <br/>
                     <br/>
                     <b>How it works</b><br/> 
                     This simulation lets you creates an ISA portfolio of 100 to 2000
                     students and tracks their status and monthly payments over time.
                     You are able to input certain variables to structure the terms 
                     of the ISA program according to a targeted ROI.
                     <br/>
                     <br/>
                     <b>How are payments decided?</b>
                     <br/>
                     Students can occupy 7 different statuses
                     <br/>
                     0. Not Graduated<br/>
                     1. Grace Period<br/>
                     2. In Repayment<br/>
                     3. In Deferment<br/>
                     4. In Delinquency<br/>
                     5. End (Term conditions reached)<br/>
                     6. End (Dropped out or Defaulted)<br/>
                     <br/>
                     Using external data (mainly from the census bureau) to initialize income distributions
                     and rates of delinquency, this simulation will transition students
                     between statuses every month according to their current income,
                     income share as well as their specific ISA term conditions. 
                     <br/>
                     <br/>
                     <b>How are ISA terms structured?</b>
                     <br/>
                     From the target ROI and length of pay term, the model calculates an income share that
                     would payout the return on investment over the course of the pay term. It does not
                     account for defaults, delinquencies or dropouts so the actual returns are always 
                     smaller than the target ROI.
                     <br/>
                     <br/>
                     <b>What assumptions are baked in?</b>
                     <br/>
                     i). Monthly likelihood of delinquency starts at a baseline level of 5% and increases by a 
                     burden factor that takes into account income share (higher income shares = higher burden), 
                     proximity to reaching any of the end conditions (closer = less burden) as well
                     as job market (recession = higher burden).
                     <br/>
                     <br/>
                     ii).Income growth is simulated as discrete occurences rather than a continuous rise.
                     Every year, the model will randomly increase the income of X% of students by the 
                     income growth rate set by the user. The percentage of students whose income has 
                     increased is determined by the job market variable as follows:<br/>
                         Recession = 70%
                         <br/>
                         Stable = 85%
                         <br/>
                         Expansion = 95%
                         <br/>
                         <br/>
                     i). Starting income and unemployment rate is set according to a distribution 
                     that approximates data from the Census Bureau.
                     
                     ''', layout=widgets.Layout(width='600px'))


In [485]:
# widget layout and reset button
layout = widgets.Layout(width='400px', height='40px')

reset_widget = widgets.Button(description='Reset', disabled=False,
                               button_style='Danger', icon='circle-o-notch')

# default widget list
output_init = widgets.Output()
output_runsim = widgets.Output()

tabput_sim = widgets.Output()
tabput_voc = widgets.Output()

numstu_widget = widgets.IntSlider(min=100, max=2000, step=100, description='Students: ', value=1000,
                                  readout_format='d', layout=layout)
paycap_widget = widgets.FloatSlider(min=1.0, max=2.5, step=0.25, description='PayCap: ', value=2.0,
                                    readout_format='.0%', layout=layout)
payterms_widget = widgets.IntSlider(min=2, max=12, step=1, description='TermsYrs: ', value=5, layout=layout)
expterms_widget = widgets.IntSlider(min=5, max=20, step=1, description='ExpYrs', value=10, layout=layout)
mindis_widget = widgets.FloatSlider(min=1000, max=5000, step=1000, description='MinDisb: ', value=2000,
                                    readout_format='$', layout=layout)
maxdis_widget = widgets.FloatSlider(min=5000, max=25000, step=1000, description='MaxDisb: ', value=20000,
                                 readout_format='$', layout=layout)
mincome_widget = widgets.IntSlider(min=20000, max=60000, step=5000, description='MinIncome: ', value=30000,
                                   readout_format = '$', layout=layout)
expm_widget = widgets.FloatSlider(min=0.0, max=1.5, step=0.1, description='TargetROI: ', value=0.5,
                                  readout_format='.0%', layout=layout)
igr_widget = widgets.FloatSlider(min=0.01, max=0.1, step=0.01, description='IncGrowthR: ', value=0.05,
                                 readout_format='.0%', layout=layout)

econ_widget = widgets.Dropdown(options=[('Recession', 0), ('Stable', 1), 
                                           ('Expansion', 2)],
                                  description='Job Market:', value=1, disabled=False)

grad_widget = widgets.FloatSlider(min=0.6, max=1.0, step=0.05, description='4YrGrad: ', value=0.7,
                                  readout_format='.0%', layout=layout)

init_widget = widgets.Button(description='Initialize Variables', disabled=False,
                               button_style='success', icon='cog')
runsim_widget = widgets.Button(description='Run Simulation', disabled=False,
                               button_style='success', icon='play')


default_list_widget = [numstu_widget, paycap_widget, payterms_widget, 
                       expterms_widget, mindis_widget, maxdis_widget,
                       mincome_widget, expm_widget, igr_widget, econ_widget,
                       grad_widget, widgets.HBox([init_widget, reset_widget])]

default_box = widgets.VBox(default_list_widget)

In [486]:
# vocational school widget list

output_init_v = widgets.Output()
output_runsim_v = widgets.Output()

vocschool_drop = widgets.Dropdown(options=[('Certificate', 0), ('Associates Degree', 1)],
                                  description='Level:', disabled=False, layout=layout)

vocfield_drop = widgets.Dropdown(options=[('Construction Management', 0), ('Nursing', 1), 
                                           ('Cosmetology', 2), ('Medical Assistant', 3), 
                                           ('Mechanic', 4), ('Dental Hygienist', 5)],
                                  description='Field:', disabled=False, layout=layout)

payterms_widget_voc = widgets.IntSlider(min=2, max=12, step=1, description='TermsYrs: ', value=5, layout=layout)

expm_widget_voc = widgets.FloatSlider(min=0.0, max=1.5, step=0.1, description='TargetROI: ', value=0.5,
                                  readout_format='.0%', layout=layout)

init_widget_voc = widgets.Button(description='Generate Terms', disabled=False,
                               button_style='success', icon='cog')

voc_list_widget = [vocschool_drop, vocfield_drop, payterms_widget_voc, 
                   expm_widget_voc, widgets.HBox([init_widget_voc, reset_widget])]

vocation_box = widgets.VBox(voc_list_widget)

In [487]:
# tabs management
sim_tab = widgets.Tab(children=[info, default_box, vocation_box])
sim_tab.set_title(0, 'Description')
sim_tab.set_title(1, 'Fund Simulation')
sim_tab.set_title(2, 'Vocational Terms')

In [ ]:
# vocational sim functions

def voc_terms_cert(field, termlen, roi):
    start_inc = 0
    disb = 0
    if field == 0: # construction management
        start_inc = 70000
        disb = 3500
    elif field == 1: # nursing
        start_inc = 70000
        disb = 3500
    elif field == 2: # cosmetology
        start_inc = 70000
        disb = 3500
    elif field == 3: # medical assistant
        start_inc = 70000
        disb = 3500
    elif field == 4: # mechanic
        start_inc = 70000
        disb = 3500
    elif field == 5: # dental hygienist
        start_inc = 70000

def voc_terms_associates(field, termlen, roi):
    start_inc = 0
    disb = 0
    if field == 0: # construction management
        start_inc = 70000
        disb = 6000
    elif field == 1: # nursing
        start_inc = 70000
        disb = 20000
    elif field == 2: # cosmetology
        start_inc = 70000
    elif field == 3: # medical assistant
        start_inc = 70000
    elif field == 4: # mechanic
        start_inc = 70000
    elif field == 5: # dental hygienist
        start_inc = 70000

        

In [ ]:
# sim functions

def starting_income(num_students):
    my_arr = []
    for i in range(num_students):
        inc_below_25 = np.random.choice(np.arange(26000, 38000, 1000))
        inc_25_to_75 = np.random.choice(np.arange(39000, 65000, 1000))
        inc_75_to_95 = np.random.choice(np.arange(66000, 84000, 1000))
        inc_95 = np.random.choice(np.arange(85000, 90000, 1000))
        my_arr.append(np.random.choice([inc_below_25, inc_25_to_75, inc_75_to_95, inc_95], p=[0.25, 0.5, 0.20, 0.05]))
    return my_arr

def income_growth(curr_inc, growth_rate):
    nogrowth = 0
    if economy == 0:
        nogrowth = 0.30
    elif economy == 1:
        nogrowth = 0.15
    elif economy == 2:
        nogrowth = 0.05
    payraise = np.random.choice([0,1], p=[nogrowth, 1-nogrowth])
    if payraise == 0:
        return curr_inc
    if payraise == 1:
        return curr_inc*(1+growth_rate)

def income_share(dis_amt, start_inc, pay_terms, exp_payout_multiple):
    inc_arr = []
    years = int(pay_terms/12)
    inc = start_inc
    for i in range(years):
        inc_arr.append(inc)
        inc = income_growth(inc, income_growth_rate)
        
    total_inc = np.sum(inc_arr)    
    mean_yr_inc = total_inc/years
    exp_payout = dis_amt*exp_payout_multiple
    
    yearly_pay = exp_payout/years
    income_sh = yearly_pay/mean_yr_inc
    return income_sh

def monthly_pay(curr_inc, inc_share):
    if curr_inc >= min_income:
        return (curr_inc*inc_share)/12
    else:
        return 0
    
def income_initialize(df):
    for i, row in df.iterrows():
        curr_inc_val = 0
        if row['Status'] == 1:
            curr_inc_val = row['Expected Starting Income']
            df.at[i,'Current Income'] = curr_inc_val
    return df

def yearly_update(time_count, df):
    if time_count == 24: # after 2 years, if student hasn't graduated, they are assumed to have dropped out
        for i, row in df.iterrows():
            if row['Status'] == 0:
                df.at[i, 'Status'] = 6
    if time_count == 12: # after 1 year, x% chance that a student who hasn't graduated will graduate
        for i, row in df.iterrows():
            if row['Status'] == 0:
                    df.at[i, 'Status'] = np.random.choice([0, 1], p=[0.5, 0.5])
    elif(time_count%12) == 0:
        for i, row in df.iterrows():
            df.at[i, 'Current Income'] = income_growth(row['Current Income'], income_growth_rate)
                
def grace_update(time_count, df):
    if (time_count%6) == 0:
        for i, row in df.iterrows():
            if row['Status'] == 1:
                df.at[i, 'Status'] = 2
                df.at[i,'Current Income'] = row['Expected Starting Income']
                
                
def status_update(df):
    global snapshot_df
    for i, row in df.iterrows():
        if row['Status'] >= 5:
            pass
        else:
            mp = round(monthly_pay(row['Current Income'], row['Income Share']))

            # income doesn't meet threshold, go to deferment
            if mp == 0:
                if row['Status'] == 0:
                    df.at[i, 'Status'] = 0
                elif row['Status'] == 2:
                    df.at[i, 'Status'] = 3
                elif row['Status'] == 3:
                    df.at[i, 'Status'] = 3
                elif row['Status'] == 4:
                    df.at[i, 'Status'] = 3

            if mp != 0 and row['Status'] == 3:
                deferrate = np.random.choice([0, 1], p=[0.05, 0.95])
                if deferrate == 1:
                    df.at[i, 'Status'] = 2

            # meets max owed cap
            if row['Pay Terms'] < 1:
                df.at[i, 'Status'] = 5
            elif row['Max Owed'] < (row['Paid Off'] + mp):
                df.at[i, 'Paid Off'] = row['Max Owed']
                df.at[i, 'Status'] = 5

            # burden-weighted monthly chance of transition from repayment to delinquency
            if row['Status'] == 2:
                defrate = 0.1
                burden = row['Income Share']+0.03
                delinquent = np.random.choice([0, 1, 2], p=[burden, defrate, 1-(burden+defrate)])
                if delinquent == 0:
                    df.at[i, 'Status'] = 4
                elif delinquent == 1:
                    df.at[i, 'Status'] = 3
                else:
                    df.at[i, 'Paid Off'] += mp
                    df.at[i, 'Pay Terms'] -= 1

            # burden-weighted monthly chance of transition from delinquency to repayment
            elif row['Status'] == 4:
                burden = row['Income Share']+0.03
                non_delinquent = np.random.choice([0, 1], p=[0.5+burden, 0.5-burden])
                if non_delinquent == 1:
                    df.at[i, 'Status'] = 2
                    df.at[i, 'Paid Off'] += mp
                    df.at[i, 'Pay Terms'] -= 1
                    
    if time_count == (pay_terms+6):
        snapshot_df = df.copy(deep=True)
        
    return df




In [ ]:
# status map: 0=student, 1=graceperiod, 2=repayment, 3=deferment, 4=delinquent, 5=end
# simulation functions

# samples creation
def samples_create():
    global total_disbursement
    status = np.zeros(num_stu)
    income = np.zeros(num_stu)
    expected_income = starting_income(num_stu)
    dis_amt = np.random.choice(np.arange(min_dis, max_dis, 500), num_stu)
    pay_terms_arr = np.full(num_stu, pay_terms, dtype=int)
    income_share_list = []
    for i in range(len(expected_income)):
        income_share_list.append(income_share(dis_amt[i], expected_income[i], pay_terms_arr[i], exp_multiple))

    max_owed = dis_amt*pay_cap
    paid_off = np.zeros(num_stu)
    students = np.stack((status, income, expected_income, dis_amt, 
                         max_owed, income_share_list, pay_terms_arr, paid_off), axis=-1)

    df = pd.DataFrame(data=students, columns=['Status', 'Current Income', 'Expected Starting Income',
                                              'Disbursement Amount', 'Max Owed', 'Income Share', 
                                              'Pay Terms', 'Paid Off'])

    df['Income Share'] = pd.Series([round(val, 4) for val in df['Income Share']], index = df.index)

    total_disbursement = df['Disbursement Amount'].sum()
    print('\n\n\n Total Disbursement Amount: ', '${:,.0f}'.format(total_disbursement))
    return df


# initial graduation update
def graduate(df):
    df['Status'] = np.transpose(np.random.choice([0, 1], num_stu, p=[1-grad_rate_4year, grad_rate_4year]))
    df = income_initialize(df)
    return df

# after grace period
def grace_period(df):
    global time_count
    for i in range(6):
        time_count += 1
        repayment_per_month.append(df['Paid Off'].sum())

    for i, row in df.iterrows():
        if row['Status'] == 1:
            inc_owed = monthly_pay(row['Current Income'], row['Income Share'])
            if inc_owed == 0:
                df.at[i, 'Status'] = 3
            else:
                df.at[i, 'Status'] = 2

    print("Months Passed : ", time_count)
    print("Repayments Array : ", [0, 0, 0, 0, 0, 0])
    return df


# simulate
def simulate(df):
    global repayment_per_month, time_count, status_count
    
    for i in range(exp_terms):
        yearly_array = []
        for j in range(12):
            time_count += 1
            grace_update(time_count, df)
            yearly_update(time_count, df)
            df = status_update(df)
            repayment_per_month.append(round(df['Paid Off'].sum()))
            
        for k in range(time_count-13, time_count-1):
            temp = repayment_per_month[k+1] - repayment_per_month[k]
            yearly_array.append(int(temp))
        print("\nMonths Passed: ", time_count)
        print("Repayments Array: ", yearly_array)
        status_count.append(df['Status'].value_counts().reindex(np.arange(6), fill_value=0))

    return df

In [ ]:
# plotting functions

def plot_returns():
    roi = (repayment_per_month/total_disbursement)-1
    
    fig_1 = plt.figure(num=None, figsize=(8,6), dpi=80)
    ax = fig_1.add_subplot(111)
    
    ax.plot(np.arange(time_count), roi)
    ax.set_xlabel('Months')
    ax._set_ylabel('Total Return on Investment')
    plt.show()
    
def plot_monthly_payments():
    global monthly_payarray
    print('\n\n\n Monthly Payments Graph')
    for i in range(len(repayment_per_month)-1):
        temp = repayment_per_month[i+1] - repayment_per_month[i]
        monthly_payarray.append(temp)

        
    fig_2 = plt.figure(num=None, figsize=(8,6), dpi=80)
    ax = fig_2.add_subplot(111)
    ax.plot(np.arange(time_count-1), monthly_payarray)
    ax.set_xlabel('Months')
    ax.set_ylabel('Dollars paid back')
    plt.show()
    
def plot_yearly_payments():
    print('\n\n\n Yearly Payments Graph')
    yield_arr = monthly_payarray/total_disbursement
    year_yield = np.add.reduceat(yield_arr, np.arange(0, len(yield_arr), 12))
    
    fig_3 = plt.figure(num=None, figsize=(8,6), dpi=80)
    ax = fig_3.add_subplot(111)
    
    ax.plot(np.arange(1, (time_count/12)+1), year_yield, marker='o', label='Payments')
    ax.set_xlabel('Year')
    ax.set_ylabel('Coupon Rate')
    plt.show()
    
    
def plot_status(stat_df):
    print('\n\n\n Yearly Status Graph')
    fig_4 = plt.figure(num=None, figsize=(8,6), dpi=80)
    ax = fig_4.add_subplot(111)
    
    years = stat_df['Year'].unique()
    
    ax.plot(years, stat_df[stat_df['Status']==0]['Count'], marker='o', label='Student')
    ax.plot(years, stat_df[stat_df['Status']==2]['Count'], marker='o', label='Repayment')
    ax.plot(years, stat_df[stat_df['Status']==3]['Count'], marker='o', label='Deferment')
    ax.plot(years, stat_df[stat_df['Status']==4]['Count'], marker='o', label='Delinquent')
    ax.plot(years, stat_df[stat_df['Status']==5]['Count'], marker='o', label='Completed')
    
    ax.set_xlabel('Year')
    ax.set_ylabel('Count of Students')
    plt.legend(loc=1)
    plt.show()    
    
# performance functions

def statusparser():
    yrcount = 1
    year_holder = [i for i in np.zeros(6, dtype=int)]
    index_holder = [i for i in np.arange(6)]
    val_holder = [i for i in np.concatenate(([num_stu],np.zeros(5, dtype=int)), axis=None)]
    for year in status_count:
        for index, values in year.iteritems():
            index_holder.append(index)
            val_holder.append(values)
            year_holder.append(yrcount)
        yrcount += 1
    
    tempdata = np.stack((year_holder, index_holder, val_holder), axis=-1)
    stat_df = pd.DataFrame(data=tempdata, columns=['Year', 'Status', 'Count'])
    return stat_df


def print_fund_performance():
    print('\n\n\n Fund Performance Summary')
    prof = (repayment_per_month[-1] - total_disbursement)
    roi = (prof/total_disbursement)
    fund_perform = pd.DataFrame(columns=['Total Amount Invested', 'Sum of Cashflows', 'Net Return ($)', 'ROI'])
    
    fund_perform.loc[0] = ['${:,.0f}'.format(total_disbursement), '${:,.0f}'.format(repayment_per_month[-1]), 
                           '${:,.0f}'.format(prof), '{:.1%}'.format(roi)]
    fund_perform = fund_perform.rename(index={0:''})
    return fund_perform


def print_yield_array():
    print('\n\n\n Yearly Cashflows')
    yield_arr = monthly_payarray/total_disbursement
    year_yield_temp = np.add.reduceat(yield_arr, np.arange(0, len(yield_arr), 12))
    year_yield = [f'{i*100:.1f}%' for i in year_yield_temp]
    
    ret_temp = np.add.reduceat(monthly_payarray, np.arange(0, len(monthly_payarray), 12))
    ret = [f'${i:,.0f}' for i in ret_temp]
    
    dat_0 = ret[:(exp_terms)]
    dat_1 = year_yield[:(exp_terms)]    
    
    yield_df = pd.DataFrame(columns=[f'Year {i}' for i in np.arange(1, (len(dat_0)+1))])
    yield_df.loc[0] = dat_0
    yield_df.loc[1] = dat_1
    
    yield_df = yield_df.rename(index={0:'Cashflows', 1:'Coupon Rate Equivalent'})
    return yield_df


def print_fund_statistics(df):
    print('\n\n\n Portfolio Statistics')
    temp_df = df.copy(deep=True)
    temp_df['Percent Paid Off'] = temp_df['Paid Off']/temp_df['Disbursement Amount'] 
    
    med_disb = '${:,.0f}'.format(temp_df['Disbursement Amount'].median())
    med_startinc = '${:,.0f}'.format(temp_df['Expected Starting Income'].median())
    med_endinc = '${:,.0f}'.format(temp_df['Current Income'].median())
    med_incshare = '{:.1%}'.format(temp_df['Income Share'].median())
    med_paidoff = '${:,.0f}'.format(temp_df['Paid Off'].median())
    med_perc_paidoff = '{:.1%}'.format(temp_df['Percent Paid Off'].median())
    
    av_disb = '${:,.0f}'.format(temp_df['Disbursement Amount'].mean())
    av_startinc = '${:,.0f}'.format(temp_df['Expected Starting Income'].mean())
    av_endinc = '${:,.0f}'.format(temp_df['Current Income'].mean())
    av_incshare = '{:.1%}'.format(temp_df['Income Share'].mean())
    av_paidoff = '${:,.0f}'.format(temp_df['Paid Off'].mean())
    av_perc_paidoff = '{:.1%}'.format(temp_df['Percent Paid Off'].mean())
    
    fundstat_df = pd.DataFrame(columns=['Disbursement', 'Expected Starting Income', 'End Income',
                                        'Income Share', 'Repaid', 'Percent of Initial Repaid'])
    fundstat_df.loc[0] = [med_disb, med_startinc, med_endinc, 
                          med_incshare, med_paidoff, med_perc_paidoff]
    fundstat_df.loc[1] = [av_disb, av_startinc, av_endinc, 
                          av_incshare, av_paidoff, av_perc_paidoff]
    
    fundstat_df = fundstat_df.rename(index={0:'Median', 1:'Mean'})
    return fundstat_df


In [ ]:
# widget functions

def reset_run_all(ev):
    output_init.clear_output(wait=False)
    output_runsim.clear_output(wait=False)
    display(Javascript('IPython.notebook.execute_all_cells()'))

def initfunc(b=None):
    global num_stu, pay_cap, pay_terms, exp_terms, exp_multiple, economy
    global min_income, min_dis, max_dis, income_growth_rate, grad_rate_4year
    
    num_stu = numstu_widget.value
    pay_cap = paycap_widget.value
    pay_terms = payterms_widget.value*12
    exp_terms = expterms_widget.value
    min_dis = mindis_widget.value
    max_dis = maxdis_widget.value
    exp_multiple = expm_widget.value+1
    economy = econ_widget.value
    min_income = mincome_widget.value
    income_growth_rate = igr_widget.value
    grad_rate_4year = grad_widget.value
    with output_init:
        
        print('\nINPUTS SELECTED')
        print('\nNumber of Students: ', num_stu)
        print('\nDisbursement Range: ', '${:,.0f}'.format(min_dis),'-', '{:,.0f}'.format(max_dis))
        print('\nPayment Cap: ', '{:.0%}'.format(pay_cap))
        print('\nNumber of Pay Terms: ', int(pay_terms/12), 'Years')
        print('\nExpiration Length: ', exp_terms, 'Years')
        print('\nTarget ROI: ', '{:.0%}'.format(exp_multiple-1))
        print('\nMinimum Income Threshold: ', '${:,.0f}'.format(min_income))
        print('\nIncome Growth Rate: ', '{:.0%}'.format(income_growth_rate))
        if economy == 0:
            print('\nJob Market: Recession')
        elif economy == 1:
            print('\nJob Market: Stable')
        else:
            print('\nJob Market: Expansion')
        print('\n4-Year Graduation Rate: ', '{:.0%}'.format(grad_rate_4year))
        display(runsim_widget)


def runsim(b=None):
    with output_runsim:
        sim_df = samples_create()
        print('\n Initializing DataFrame of %d Students' %num_stu)
        display(sim_df.head(6))
        sim_df = graduate(sim_df)
        sim_df = grace_period(sim_df)
        sim_df = simulate(sim_df)
        print('\n\n\n Final Dataframe')
        display(sim_df.head(6))
        display(print_fund_statistics(sim_df))
        plot_monthly_payments()
        plot_yearly_payments()
        plot_status(statusparser())
        display(print_fund_performance())
        display(print_yield_array())
        display(reset_widget)
        
        
def voc_gen(b=None):
    global v_level, v_field, v_terms, v_target_roi
    
    v_level = vocschool_drop.value
    v_field = vocfield_drop.value
    v_terms = payterms_widget_voc.value*12
    v_target_roi = expm_widget_voc.value+1
    if v_level == 0:
        x = voc_terms_cert(v_field, v_terms, v_target_roi)
    elif v_level == 1:
        x = voc_terms_associates(v_field, v_terms, v_target_roi)

In [ ]:
# button function calls
reset_widget.on_click(reset_run_all)
init_widget.on_click(initfunc)
runsim_widget.on_click(runsim)
init_widget_voc.on_click(voc_gen)

# display widgets
display(sim_tab)
display(output_init)
display(output_runsim)